## RNN

In [1]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train / 255
x_test = x_test / 255
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [2]:
# 모델 생성
X = tf.keras.Input(shape=[28, 28])

# cell = tf.keras.layers.SimpleRNNCell(32)
# H = tf.keras.layers.RNN(cell)(X)
H = tf.keras.layers.SimpleRNN(32, return_sequences=False)(X)

Y = tf.keras.layers.Dense(10, activation="softmax")(H)

model = tf.keras.Model(X, Y)
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics="accuracy"
)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                1952      
                                                                 
 dense (Dense)               (None, 10)                330       
                                                                 
Total params: 2,282
Trainable params: 2,282
Non-trainable params: 0
_________________________________________________________________


In [3]:
model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.1)

Epoch 1/10
422/422 [==============================] - 3s 7ms/step - loss: 1.2539 - accuracy: 0.5833 - val_loss: 0.7118 - val_accuracy: 0.7707
Epoch 2/10
422/422 [==============================] - 2s 6ms/step - loss: 0.6973 - accuracy: 0.7736 - val_loss: 0.5146 - val_accuracy: 0.8425
Epoch 3/10
422/422 [==============================] - 2s 6ms/step - loss: 0.5338 - accuracy: 0.8360 - val_loss: 0.3779 - val_accuracy: 0.8898
Epoch 4/10
422/422 [==============================] - 2s 5ms/step - loss: 0.4262 - accuracy: 0.8729 - val_loss: 0.3262 - val_accuracy: 0.9052
Epoch 5/10
422/422 [==============================] - 2s 5ms/step - loss: 0.3675 - accuracy: 0.8911 - val_loss: 0.2730 - val_accuracy: 0.9202
Epoch 6/10
422/422 [==============================] - 2s 5ms/step - loss: 0.3259 - accuracy: 0.9042 - val_loss: 0.3406 - val_accuracy: 0.8985
Epoch 7/10
422/422 [==============================] - 2s 4ms/step - loss: 0.3011 - accuracy: 0.9138 - val_loss: 0.2399 - val_accuracy: 0.9297
Epoch 

### 삼성전자 주가 예측
https://github.com/FinanceData/FinanceDataReader

In [ ]:
!pip install finance-datareader

In [ ]:
import FinanceDataReader as fdr

# 삼성전자(005930) 전체 (1996-11-05 ~ 현재)
dataset = fdr.DataReader('005930')
dataset = dataset[['Open', 'High', 'Low', 'Close', 'Volume']]
dataset = (dataset - dataset.min()) / (dataset.max() - dataset.min())
dataset = dataset.values
print(dataset.shape)
dataset[:5]

In [ ]:
import numpy as np

# 입력 데이터 준비
x_train = np.array([dataset[i:i+7] for i in range(len(dataset) - 7)])
y_train = np.array([dataset[i+7, 3] for i in range(len(dataset) - 7)])
print(x_train.shape, y_train.shape)

# train, test 분리
x_train, x_test = x_train[:-500], x_train[-500:]
y_train, y_test = y_train[:-500], y_train[-500:]
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
X = tf.keras.Input(shape=[7, 5])
cell = tf.keras.layers.LSTMCell(32)
H = tf.keras.layers.RNN(cell)(X)
Y = tf.keras.layers.Dense(1)(H)

model = tf.keras.Model(X, Y)
model.compile(loss="mse", optimizer="adam")
model.summary()

In [ ]:
model.fit(x_train, y_train, epochs=50, batch_size=128)

In [ ]:
import matplotlib.pyplot as plt

y_pred = model.predict(x_test)

plt.figure(figsize=(10, 5))
plt.plot(y_pred)
plt.plot(y_test)
plt.legend(['pred', 'real'])
plt.show()